In [4]:
import pandas as pd
import fnmatch, os
import openpyxl

In [1]:
path, evaluation ='../../../data/raw files/', 'Evaluations/'
%ls '$path'

ls: $path: No such file or directory


In [ ]:
pd.set_option("max_rows", None)

In [89]:
aws_keep_rows, tf_keep_rows, k8_keep_rows = ["Total", "Overall"], ["Total", "Overall"], ["Category", "Total"]
count = 0

aws_service = [
    'EC2 (2)', 
    'S3 Arch (0.5)', 
    'S3 Access (0.5)', 
    'S3 Classes (0.5)', 
    'S3 DR (0.5)', 'IAM (1)', 
    'Networking (2)', 
    'Net. App. Reliability (2)',
    'Overall'
  ]

tf_v1_service = [
  'Process',
  'Data types',
  'Organizing resources',
  'Values to terraform',
  'Environment',
  'TF Best practices',
  'Sensitive information',
  'Configuration drift',
  'Count and ForEach',
  'Secrets and States',
  'Overall'
  ]

tf_v2_service = [
  'Process',
  'Organizing resources',
  'Values to terraform',
  'Environment',
  'TF Best practices',
  'Sensitive information',
  'Configuration drift',
  'Count and ForEach',
  'Secrets and States',
  'Overall'
  ]

k8_v1_service = [
  'Core Concepts', #1
  'Workload Management', #2
  'Services & Networking: Service',#3
  'Services & Networking: Ingress',#4
  'Architecture: Control Plane',#5
  'Architecture: Auto Scaling'#6
  ]

k8_v2_service = [
  'Architecture: Auto Scaling', #Architecture',#6
  'Services & Networking: Service',#'Services',#3
  'Core Concepts', #'PODs',#1
  'Workload Management', #'Workload Resources', #2
  'Services & Networking: Service', #'POD Health Checks',#3
  'DROP',#'Configmaps / Secrets',#9
  'Architecture: Control Plane',#'Scheduling',#5
  'Workload Management',#'Volume Provisioning',#2
  'DROP',#'RBAC',#7
  'DROP',#'Service Discovery',#8
  'Services & Networking: Ingress'#'Ingress'#4
  ]

k8_v3_service = [
  'Architecture: Auto Scaling', #Architecture',#6
  'Services & Networking: Service',#'Services',#3
  'Core Concepts', #'PODs',#1
  'Workload Management', #'Workload Resources', #2
  'Services & Networking: Service', #'POD Health Checks',#3
  'DROP',#'Configmaps / Secrets',#9
  'Architecture: Control Plane',#'Scheduling',#5
  'Workload Management',#'Volume Provisioning',#2
  'DROP',#'RBAC',#7
  'Services & Networking: Ingress',#'Ingress'#4
  'DROP',#'Service Discovery',#8
  'Architecture: Auto Scaling'#'Autoscaling Tier III'    #6
  ]


In [90]:
df, aws2, tf2, k8_2 = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

for file in os.listdir(path + evaluation): 
  if fnmatch.fnmatch(file, "*AWS*" ):
    count += 1
    aws = pd.read_excel ( path + evaluation + file )
    print ("{}, numero: {}".format(file, count))
    employee_hash = file [ : file.find('-') ] # reading hash
    aws = aws[aws.columns[0:4]]
    aws.columns = ['Service', 'Category', 'Score', 'TotalScore'] #renaming columns
    aws.drop(aws [~aws['Category'].isin(aws_keep_rows)].index, axis = 0, inplace = True) # only keeps total and overall rows
    aws['Service'] = aws_service # renaming the row for each service category
    aws['NetScore'] = round(((aws['Score'] / aws['TotalScore']) * 100).astype(float), 2)
    aws.drop (['Category', 'Score', 'TotalScore'], axis = 1, inplace = True)
    aws.insert(loc = 0, column = 'Employee', value = employee_hash)
    aws2 = aws2.append (aws)
    # aws2 = aws2.iloc[:, [0, 1, 2]]
    aws2.reset_index(0)

  if fnmatch.fnmatch(file, "*Terraform*" ):
    count += 1
    tf = pd.read_excel ( path + evaluation + file )
    print ("{}, numero: {}".format(file, count))
    employee_hash = file [ : file.find('-') ] # reading hash
    tf = tf[tf.columns[0:4]]
    tf.columns = ['Service', 'Category', 'Score', 'TotalScore'] #renaming columns
    tf.drop(tf [~tf['Category'].isin(tf_keep_rows)].index, axis = 0, inplace = True) # only keeps total and overall rows
    if tf.loc[tf['TotalScore'] == 9.75].any().any():
      tf['Service'] = tf_v1_service # renaming the row for each service category
    elif tf.loc[tf['TotalScore'] == 8.25].any().any():
      tf['Service'] = tf_v2_service # renaming the row for each service category
    tf.drop(tf [tf['Service'].isin(['Data types', 'Overall'])].index, axis = 0, inplace = True) # only keeps total and overall rows
    tf['NetScore'] = round(((tf['Score'] / tf['TotalScore']) * 100).astype(float), 2)
    tf.drop (['Category','Score', 'TotalScore'], axis = 1, inplace = True)
    tf.insert(loc = 0, column = 'Employee', value = employee_hash)
    tf2 = tf2.append(tf)
    tf2.reset_index(0)

  if fnmatch.fnmatch(file, "*Kubernetes*" ):
    count += 1
    k8 = pd.read_excel ( path + evaluation + file )
    print ("{}, numero: {}".format(file, count))
    employee_hash = file [ : file.find('-') ] # reading hash
    try:
      k8.drop(k8 [~k8['KUBERNETES EVALUATION'].isin(k8_keep_rows)].index, axis = 0, inplace = True) # only keeps total and overall rows
    except:
      flag = 0
      pass
    try:
      k8.drop(k8 [~k8['Unnamed: 2'].isin(k8_keep_rows)].index, axis = 0, inplace = True) # only keeps total and overall rows
    except:
      flag = 1
      pass

    if flag == 0:
      k8 = k8.iloc[:, [3, 5, 12]]
      k8.columns = ['Service', 'TotalScore', 'Score'] #renaming columns
      k8['Service'] = k8_v1_service
    if flag == 1:
      k8 = k8.iloc[:, [3, 4, 5]]
      k8.columns = ['Score', 'TotalScore', 'Service'] #renaming columns 
      if len(k8) == 12:
        k8['Service'] =  k8_v3_service
      elif len(k8) == 11:
        k8['Service'] =  k8_v2_service
    k8 = k8 [k8.Service != 'DROP']
    k8.insert(loc = 0, column = 'Employee', value = employee_hash)
    k8 = (k8.groupby(['Employee','Service']).sum().reset_index() )
    k8['NetScore'] = round(((k8['Score'] / k8['TotalScore']) * 100).astype(float), 2)
    k8.drop (['Score', 'TotalScore'], axis = 1, inplace = True)
    k8_2 = k8_2.append(k8)
    k8_2.reset_index(0)

tf2 = tf2.groupby(['Employee', 'Service']).sum().transpose().stack(0).reset_index()
aws2 = aws2.groupby(['Employee', 'Service']).sum().transpose().stack(0).reset_index()
k8_2 = k8_2.groupby(['Employee', 'Service']).sum().transpose().stack(0).reset_index()
df = pd.merge(aws2, tf2, on=['Employee'], how='left').merge(k8_2, on=['Employee'], how='left')

df.drop (['level_0', 'level_0_x','level_0_y'] , axis = 1, inplace = True)
df.reset_index(drop = True, inplace=True)
df['Overall'] = df.sum(axis=1) / (len(df.columns) - df.isnull().sum(axis=1))

# k8_2.head(50)

d90c3b - AWS.xlsx, numero: 1
8fabca - Kubernetes.xlsx, numero: 2
e4b911 - Terraform.xlsx, numero: 3
8a67a5 - Terraform.xlsx, numero: 4
f9e83e - Kubernetes.xlsx, numero: 5
4c72ef - AWS.xlsx, numero: 6
e0775f - AWS.xlsx, numero: 7
3e3d93 - AWS.xlsx, numero: 8
a24634 - AWS.xlsx, numero: 9
f9e83e - Terraform.xlsx, numero: 10
3fc03e - AWS.xlsx, numero: 11
a15401 - AWS.xlsx, numero: 12
b142dd - Kubernetes.xlsx, numero: 13
eac4c8 - AWS.xlsx, numero: 14
ab72d4 - AWS.xlsx, numero: 15
eb6801 - AWS.xlsx, numero: 16
64e8c8 - Kubernetes.xlsx, numero: 17
f03dce - AWS.xlsx, numero: 18
a3f478 - Kubernetes.xlsx, numero: 19
bde670 - Kubernetes.xlsx, numero: 20
92bfd3 - Kubernetes.xlsx, numero: 21
fa8416 - AWS.xlsx, numero: 22
c450c7 - Terraform.xlsx, numero: 23
da6e57 - Kubernetes.xlsx, numero: 24
cec008 - AWS.xlsx, numero: 25
2d56ea - AWS.xlsx, numero: 26
0f955f - AWS.xlsx, numero: 27
f2f2e2 - Terraform.xlsx, numero: 28
88b467 - Terraform.xlsx, numero: 29
3651e3 - AWS.xlsx, numero: 30
b2ef57 - AWS.xlsx

/var/folders/gd/xh4p1yxj22l01dr06998m3hw0000gp/T/ipykernel_98531/405244592.py:81: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['Overall'] = df.sum(axis=1 ) / (len(df.columns) - df.isnull().sum(axis=1))


In [92]:
df.head(5)
csv_file = ('Results for {}.csv' .format(len(df[:])) )
df.to_csv(path+csv_file, sep=',', index = False)